In [1]:
import gc
import os
import re
import time
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from collections import defaultdict

In [2]:
df = pd.read_csv('Product Analyst Test Task Data - Task 2.csv')
df

,user_id,event_timestamp,first_purchase_time,cohort_week,product_id,lifetime_months
0,95287b91-85af-4174-9986-aeafb3853bb6,2023-09-21 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,6
1,95287b91-85af-4174-9986-aeafb3853bb6,2023-03-25 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,0
2,95287b91-85af-4174-9986-aeafb3853bb6,2023-04-24 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,1
3,95287b91-85af-4174-9986-aeafb3853bb6,2023-05-24 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,2
4,95287b91-85af-4174-9986-aeafb3853bb6,2023-06-23 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,3
...,...,...,...,...,...,...
1195,8256ba89-6e57-4119-85a7-13aa868acfd1,2023-05-07 10:41:48,2023-05-07 10:41:48,2023-04-30,monthly.12.49,0
1196,058ba689-fe24-461e-a74e-a86b7603c1ec,2023-05-07 12:30:15,2023-05-07 12:30:15,2023-04-30,monthly.12.49,0
1197,058ba689-fe24-461e-a74e-a86b7603c1ec,2023-06-06 12:30:15,2023-05-07 12:30:15,2023-04-30,monthly.12.49,1
1198,fd1c529d-6791-4a5b-a84f-6acadd9e1e98,2023-05-07 14:00:39,2023-05-07 14:00:39,2023-04-30,monthly.12.49,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   user_id              1200 non-null   object
 1   event_timestamp      1200 non-null   object
 2   first_purchase_time  1200 non-null   object
 3   cohort_week          1200 non-null   object
 4   product_id           1200 non-null   object
 5   lifetime_months      1200 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 56.4+ KB


In [4]:
df.describe(include = 'all')

,user_id,event_timestamp,first_purchase_time,cohort_week,product_id,lifetime_months
count,1200,1200,1200,1200,1200,1200.000000
unique,561,1198,559,9,1,NaN
top,9070dd82-0b44-4c42-8c6f-76fdccf1d45f,2023-03-24 11:56:33,2023-03-06 13:54:00,2023-03-05,monthly.12.49,NaN
freq,8,2,8,197,1200,NaN
mean,NaN,NaN,NaN,NaN,NaN,1.430833
std,NaN,NaN,NaN,NaN,NaN,1.787329
min,NaN,NaN,NaN,NaN,NaN,0.000000
25%,NaN,NaN,NaN,NaN,NaN,0.000000
50%,NaN,NaN,NaN,NaN,NaN,1.000000
75%,NaN,NaN,NaN,NaN,NaN,2.000000


In [5]:
df.isnull().sum()

user_id                0
event_timestamp        0
first_purchase_time    0
cohort_week            0
product_id             0
lifetime_months        0
dtype: int64

In [6]:
df['revenue'] = df['product_id'].str.findall('\d\d\.\d+')
df

,user_id,event_timestamp,first_purchase_time,cohort_week,product_id,lifetime_months,revenue
0,95287b91-85af-4174-9986-aeafb3853bb6,2023-09-21 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,6,[12.49]
1,95287b91-85af-4174-9986-aeafb3853bb6,2023-03-25 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,0,[12.49]
2,95287b91-85af-4174-9986-aeafb3853bb6,2023-04-24 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,1,[12.49]
3,95287b91-85af-4174-9986-aeafb3853bb6,2023-05-24 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,2,[12.49]
4,95287b91-85af-4174-9986-aeafb3853bb6,2023-06-23 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,3,[12.49]
...,...,...,...,...,...,...,...
1195,8256ba89-6e57-4119-85a7-13aa868acfd1,2023-05-07 10:41:48,2023-05-07 10:41:48,2023-04-30,monthly.12.49,0,[12.49]
1196,058ba689-fe24-461e-a74e-a86b7603c1ec,2023-05-07 12:30:15,2023-05-07 12:30:15,2023-04-30,monthly.12.49,0,[12.49]
1197,058ba689-fe24-461e-a74e-a86b7603c1ec,2023-06-06 12:30:15,2023-05-07 12:30:15,2023-04-30,monthly.12.49,1,[12.49]
1198,fd1c529d-6791-4a5b-a84f-6acadd9e1e98,2023-05-07 14:00:39,2023-05-07 14:00:39,2023-04-30,monthly.12.49,0,[12.49]


In [7]:
ad=[]
for z in range(len(df)):
    ad.append([float(i) for i in df['revenue'][z]])
df['revenue_v'] = np.array(ad,dtype=float)
df

,user_id,event_timestamp,first_purchase_time,cohort_week,product_id,lifetime_months,revenue,revenue_v
0,95287b91-85af-4174-9986-aeafb3853bb6,2023-09-21 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,6,[12.49],12.49
1,95287b91-85af-4174-9986-aeafb3853bb6,2023-03-25 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,0,[12.49],12.49
2,95287b91-85af-4174-9986-aeafb3853bb6,2023-04-24 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,1,[12.49],12.49
3,95287b91-85af-4174-9986-aeafb3853bb6,2023-05-24 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,2,[12.49],12.49
4,95287b91-85af-4174-9986-aeafb3853bb6,2023-06-23 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,3,[12.49],12.49
...,...,...,...,...,...,...,...,...
1195,8256ba89-6e57-4119-85a7-13aa868acfd1,2023-05-07 10:41:48,2023-05-07 10:41:48,2023-04-30,monthly.12.49,0,[12.49],12.49
1196,058ba689-fe24-461e-a74e-a86b7603c1ec,2023-05-07 12:30:15,2023-05-07 12:30:15,2023-04-30,monthly.12.49,0,[12.49],12.49
1197,058ba689-fe24-461e-a74e-a86b7603c1ec,2023-06-06 12:30:15,2023-05-07 12:30:15,2023-04-30,monthly.12.49,1,[12.49],12.49
1198,fd1c529d-6791-4a5b-a84f-6acadd9e1e98,2023-05-07 14:00:39,2023-05-07 14:00:39,2023-04-30,monthly.12.49,0,[12.49],12.49


In [8]:
df['Total_revenue'] = df['revenue_v']*df['lifetime_months']
df

,user_id,event_timestamp,first_purchase_time,cohort_week,product_id,lifetime_months,revenue,revenue_v,Total_revenue
0,95287b91-85af-4174-9986-aeafb3853bb6,2023-09-21 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,6,[12.49],12.49,74.94
1,95287b91-85af-4174-9986-aeafb3853bb6,2023-03-25 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,0,[12.49],12.49,0.00
2,95287b91-85af-4174-9986-aeafb3853bb6,2023-04-24 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,1,[12.49],12.49,12.49
3,95287b91-85af-4174-9986-aeafb3853bb6,2023-05-24 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,2,[12.49],12.49,24.98
4,95287b91-85af-4174-9986-aeafb3853bb6,2023-06-23 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,3,[12.49],12.49,37.47
...,...,...,...,...,...,...,...,...,...
1195,8256ba89-6e57-4119-85a7-13aa868acfd1,2023-05-07 10:41:48,2023-05-07 10:41:48,2023-04-30,monthly.12.49,0,[12.49],12.49,0.00
1196,058ba689-fe24-461e-a74e-a86b7603c1ec,2023-05-07 12:30:15,2023-05-07 12:30:15,2023-04-30,monthly.12.49,0,[12.49],12.49,0.00
1197,058ba689-fe24-461e-a74e-a86b7603c1ec,2023-06-06 12:30:15,2023-05-07 12:30:15,2023-04-30,monthly.12.49,1,[12.49],12.49,12.49
1198,fd1c529d-6791-4a5b-a84f-6acadd9e1e98,2023-05-07 14:00:39,2023-05-07 14:00:39,2023-04-30,monthly.12.49,0,[12.49],12.49,0.00


In [9]:
df.loc[(df['Total_revenue'] == 0),'Total_revenue'] =df['revenue_v']
df

,user_id,event_timestamp,first_purchase_time,cohort_week,product_id,lifetime_months,revenue,revenue_v,Total_revenue
0,95287b91-85af-4174-9986-aeafb3853bb6,2023-09-21 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,6,[12.49],12.49,74.94
1,95287b91-85af-4174-9986-aeafb3853bb6,2023-03-25 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,0,[12.49],12.49,12.49
2,95287b91-85af-4174-9986-aeafb3853bb6,2023-04-24 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,1,[12.49],12.49,12.49
3,95287b91-85af-4174-9986-aeafb3853bb6,2023-05-24 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,2,[12.49],12.49,24.98
4,95287b91-85af-4174-9986-aeafb3853bb6,2023-06-23 4:44:40,2023-03-25 4:44:40,2023-03-19,monthly.12.49,3,[12.49],12.49,37.47
...,...,...,...,...,...,...,...,...,...
1195,8256ba89-6e57-4119-85a7-13aa868acfd1,2023-05-07 10:41:48,2023-05-07 10:41:48,2023-04-30,monthly.12.49,0,[12.49],12.49,12.49
1196,058ba689-fe24-461e-a74e-a86b7603c1ec,2023-05-07 12:30:15,2023-05-07 12:30:15,2023-04-30,monthly.12.49,0,[12.49],12.49,12.49
1197,058ba689-fe24-461e-a74e-a86b7603c1ec,2023-06-06 12:30:15,2023-05-07 12:30:15,2023-04-30,monthly.12.49,1,[12.49],12.49,12.49
1198,fd1c529d-6791-4a5b-a84f-6acadd9e1e98,2023-05-07 14:00:39,2023-05-07 14:00:39,2023-04-30,monthly.12.49,0,[12.49],12.49,12.49


In [22]:
df.to_csv('Test Task Data - Task 22.csv')

In [10]:
sale=df.pivot_table(index = ['lifetime_months'], columns=['cohort_week'],values='Total_revenue',aggfunc='sum',fill_value=0).reset_index()
sale


cohort_week,lifetime_months,2023-03-05,2023-03-12,2023-03-19,2023-03-26,2023-04-02,2023-04-09,2023-04-16,2023-04-23,2023-04-30
0,0,1099.12,861.81,1049.16,624.50,449.64,537.07,699.44,836.83,849.32
1,1,412.17,274.78,324.74,274.78,187.35,212.33,199.84,287.27,349.72
2,2,499.60,449.64,474.62,399.68,224.82,349.72,199.84,349.72,574.54
3,3,786.87,562.05,636.99,374.70,262.29,412.17,187.35,337.23,599.52
4,4,549.56,499.60,599.52,449.64,249.80,499.60,199.84,249.80,599.52
5,5,686.95,437.15,686.95,499.60,187.35,499.60,312.25,312.25,249.80
6,6,524.58,524.58,524.58,449.64,149.88,524.58,74.94,0.00,0.00
7,7,524.58,174.86,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [18]:
X=sale[['2023-03-05','2023-03-12', '2023-03-19','2023-03-26','2023-04-02','2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30']]
y=sale[['CLV']]
#split training set and test set
from sklearn.linear_model import LinearRegression
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)
# import model
from sklearn.linear_model import LinearRegression

# instantiate
linreg = LinearRegression()

# fit the model to the training data (learn the coefficients)
linreg.fit(X_train, y_train)

# make predictions on the testing set
y_pred = linreg.predict(X_test)


In [17]:
sale['CLV']=sale.iloc[:,2:].mean(axis=1)
sale

cohort_week,lifetime_months,2023-03-05,2023-03-12,2023-03-19,2023-03-26,2023-04-02,2023-04-09,2023-04-16,2023-04-23,2023-04-30,CLV
0,0,1099.12,861.81,1049.16,624.50,449.64,537.07,699.44,836.83,849.32,738.47125
1,1,412.17,274.78,324.74,274.78,187.35,212.33,199.84,287.27,349.72,263.85125
2,2,499.60,449.64,474.62,399.68,224.82,349.72,199.84,349.72,574.54,377.82250
3,3,786.87,562.05,636.99,374.70,262.29,412.17,187.35,337.23,599.52,421.53750
4,4,549.56,499.60,599.52,449.64,249.80,499.60,199.84,249.80,599.52,418.41500
5,5,686.95,437.15,686.95,499.60,187.35,499.60,312.25,312.25,249.80,398.11875
6,6,524.58,524.58,524.58,449.64,149.88,524.58,74.94,0.00,0.00,281.02500
7,7,524.58,174.86,0.00,0.00,0.00,0.00,0.00,0.00,0.00,21.85750


In [13]:
# print the intercept and coefficients
print(linreg.intercept_)
print(linreg.coef_)


[1.91508494]
[[0.00559157 0.09727319 0.14770577 0.12564203 0.07207888 0.1101897
  0.13939614 0.11525346 0.15025314]]


In [20]:
#Building the Decision Tree Model on our dataset
from sklearn.tree import DecisionTreeRegressor
DT_model = DecisionTreeRegressor(max_depth=5).fit(X_train, y_train)
DT_predict = DT_model.predict(X_test) #Predictions on Testing data
print(DT_predict)


[21.8575 21.8575]


In [14]:
from sklearn import metrics

# compute the R Square for model
print("R-Square:",metrics.r2_score(y_test, y_pred))


R-Square: 0.9686430977768798
